Overview
--------

Extracting building footprints from LiDAR derived elevation models, NAIP, and thematic mask layers.


Input Data
----------

The thematic data include a subset of roads, hydroline, and hydropoly datasets for Jefferson county WV and Washington county MD.

The input LAZ file in sample data is in UTM zone 18 N.  The derivatative DSM, DTM, and DEM were created with the commands outlined below the `lidar-processing` utility should handle point filtering but currently lacks that functionality.  `las2las` was obtained from a homebrewed install of libLAS.  The input LiDAR data has already been de-noised.

Surface models generated from LiDAR have different names depending on who you ask so for the sake of this exercis they are defined as follows:

1. Digital Surface Model (DSM) - Gridded first return
2. Digital Terrain Model (DTM) - Gridded last return
3. Digital Elevation Model (DEM) - Gridded ground returns/classification

A Normalized Digital Terrain Model is DTM - DEM, which leaves roughly absolute height above ground for the remaining pixels and eliminates most vegetation.  It is immediately clipped to > 3m in order to eliminate ground pixel noise and any building we really care about is at least 3 meters tall.
    
    # DTM
    las2las --last-return-only \
        -i sample-data/VA-WV-MD_FEMA_Region3_UTM18_2012_000842.laz \
        -o sample-data/last.las
    ./lidar-processing.py \
        sample-data/last.las \
        sample-data/DTM.tif \
        -tr 1 1 \
        -co COMPRESS=DEFLATE -co TILED=YES -co PREDICTOR=3 -co ZLEVEL=9 \
        -crs EPSG:32618 \
        --interpolation nearest

    # DEM
    las2las --keep-classes 2 \
        -i sample-data/VA-WV-MD_FEMA_Region3_UTM18_2012_000842.laz \
        -o sample-data/ground.las
    ./lidar-processing.py \
        sample-data/ground.las \
        sample-data/DEM.tif \
        -tr 1 1 \
        -co COMPRESS=DEFLATE -co TILED=YES -co PREDICTOR=3 -co ZLEVEL=9 \
        -crs EPSG:32618 \
        --interpolation nearest
        
    # DSM - not needed for this exercise
    las2las --first-return-only \
        -i sample-data/VA-WV-MD_FEMA_Region3_UTM18_2012_000842.laz \
        -o sample-data/first.las
    ./lidar-processing.py \
        sample-data/first.las \
        sample-data/DSM.tif \
        -tr 1 1 \
        -co COMPRESS=DEFLATE -co TILED=YES -co PREDICTOR=3 -co ZLEVEL=9 \
        -crs EPSG:32618 \
        --interpolation nearest
        

###Imports

In [1]:
%matplotlib inline
import os
import fiona
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.features import rasterize
from shapely.geometry import asShape

###Identify all the needed files, make sure they exist

Cache the DTM raster's metadata so we can extract information from it later.

In [2]:
dtm_path = 'sample-data/DTM.tif'
dem_path = 'sample-data/DEM.tif'

# Keys are paths and vals are buffer distances
thematic_mask_paths = {
    'sample-data/tl_2014_24043_areawater.geojson': 0,
    'sample-data/tl_2014_24_prisecroads.geojson': 3,
    'sample-data/tl_2014_54037_areawater.geojson': 0,
    'sample-data/tl_2014_54037_linearwater.geojson': 1,
    'sample-data/tl_2014_54037_roads.geojson': 3
}

# Define some additional commonly used variables
with rasterio.drivers(), fiona.drivers():
    with rasterio.open(dtm_path) as src:
        dtm_meta = src.meta.copy()
        dtm_shape = src.shape
        dtm_nodata = src.meta['nodata']
        dtm_dtype = src.meta['dtype']
        dtm_affine = src.meta['affine']

###Create a mask layer from the thematic datasets

In [3]:
mask_layer = np.full(dtm_shape, 0, dtm_dtype)
for mask_path, buff_dist in thematic_mask_paths.items():
    with fiona.open(mask_path) as src:
        mask_layer = rasterize(
            shapes=[asShape(f['geometry']).buffer(buff_dist) for f in src],
            out_shape=mask_layer,
            fill=1,
            out=mask_layer,
            transform=dtm_affine,
            all_touched=True,
            default_value=0,
            dtype='uint8'
        )
        
            
# Burn all geometries into a single numpy array
# mask_layer = rasterize(
#     mask_geometries,
#     out_shape=(dtm_meta['height'], dtm_meta['width']),
#     transform=dtm_meta['affine'],
#     all_touched=True,
#     dtype='uint8'
# ).astype(np.bool)

FionaValueError: No dataset found at path 'sample-data/tl_2014_54037_roads.geojson' using drivers: *

In [6]:
# Generate a Normalized DTM by subtracting the DEM from the DTM
with rasterio.open(dtm_path) as dtm, rasterio.open(dem_path) as dem:
    ndtm = dtm.read_band(1) - dem.read_band(1)

###Display layers

In [ ]:
print("Thematic mask")
plt.imshow(mask_layer)
plt.show()

In [ ]:
print("nDTM:")
plt.imshow(ndtm)
plt.set_cmap('YlGnBu_r')
plt.show()

In [ ]:
print("DTM:")
with rasterio.open(dtm_path) as dtm:
    plt.imshow(dtm.read_band(1))
    plt.set_cmap('YlGnBu_r')
    plt.show()

In [ ]:
print("DEM:")
with rasterio.open(dem_path) as dem:
    plt.imshow(dem.read_band(1))
    plt.set_cmap('YlGnBu_r')
    plt.show()